In [28]:
import re
import spacy
from spacy.matcher import Matcher
import fitz

In [29]:
# def remove_extra_spaces(text):
#     return re.sub(r"\s+", " ", text.strip())

In [30]:
# Carga el modelo de spaCy (si vas a usar spaCy en otras partes)
nlp = spacy.load("es_core_news_lg")
matcher = Matcher(nlp.vocab) 

In [31]:
def extraer_metadatos_gaceta(ruta_pdf):
    """
    Extrae metadatos de una gaceta del Congreso.
    """
    metadatos = {}

    texto_completo = preprocesar_texto(ruta_pdf)

            # 1. Nombre Principal
    if re.search(r"(?i)GACETA\s+DEL\s+CONGRESO", texto_completo):
        metadatos["nombre"] = "Gaceta del Congreso"

    # --- ISSN ---
    metadatos["issn"] = extraer_con_regex(r"I\s*S\s*S\s*N\s*(0\s*1\s*2\s*3\s*-\s*9\s*0\s*6\s*6)", texto_completo)
    if metadatos["issn"]:
        metadatos["issn"] = metadatos["issn"].replace(" ", "")

    # --- 2. Fecha ---
    patron_fecha = r"(?i)(lunes|martes|miércoles|jueves|viernes|sábado|domingo),\s*(\d{1,2})\s+de\s+(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\s+de\s+(\d{4})"
    coincidencia_fecha = re.search(patron_fecha, texto_completo)
    if coincidencia_fecha:
        meses = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,
                    "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12}
        dia = int(coincidencia_fecha.group(2))
        mes = meses[coincidencia_fecha.group(3).lower()]
        anio = int(coincidencia_fecha.group(4))
        metadatos["fecha"] = f"{dia:02d}/{mes:02d}/{anio}"

    # 3. Entidad (Senado/Cámara)
    match_entity = re.search(r"(?i)(SENADO Y CÁMARA|SENADO|CÁMARA)", texto_completo)
    if match_entity:
        entity = match_entity.group(1).strip()
        if entity == "SENADO Y CÁMARA":
            metadatos["entidades"] = ["SENADO Y CAMARA", "SENADO", "CÁMARA"]
        else:
            metadatos["entidades"] = [entity]

    # --- 4. Año (Números Romanos) y Número de Publicación ---
    patron_anio_numero = r"AÑO\s+([MDCLXVI]+)\s*-\s*N[°º]\s*(\d+)"
    coincidencia_anio_numero = re.search(patron_anio_numero, texto_completo, re.IGNORECASE)
    if coincidencia_anio_numero:
        anio_romano = coincidencia_anio_numero.group(1).strip()
        numero_publicacion = coincidencia_anio_numero.group(2).strip()

        metadatos["anio_romano"] = anio_romano
        metadatos["anio"] = romano_a_entero(anio_romano)
        metadatos["numero_publicacion"] = int(numero_publicacion)

    # --- 5. Rama Legislativa ---
    pattern_branch = r"(?i)RAMA\s+LEGISLATIVA\s+DEL\s+PODER\s+PÚBLICO"
    if re.search(pattern_branch, texto_completo):
        metadatos["rama"] = "Rama Legislativa del Poder Público"

    # --- 6. Directores (Enfoque Híbrido Robusto) ---
    metadatos["directores"] = {"senado": {}, "camara": {}}
    patron_directores_inicio = r"DIRECTORES:"
    coincidencia_inicio = re.search(patron_directores_inicio, texto_completo, re.IGNORECASE)

    if coincidencia_inicio:
        inicio_seccion = coincidencia_inicio.end()
        patron_directores_fin = r"(?:SECRETARIO GENERAL DEL SENADO|SECRETARIO GENERAL DE LA CÁMARA|RAMA LEGISLATIVA)"
        coincidencia_fin = re.search(patron_directores_fin, texto_completo[inicio_seccion:], re.IGNORECASE)

        if coincidencia_fin:
            fin_seccion = inicio_seccion + coincidencia_fin.start()
            texto_directores = texto_completo[inicio_seccion:fin_seccion]


            lineas = texto_directores.splitlines()

            nombre_senado = ""
            cargo_senado = ""
            nombre_camara = ""
            cargo_camara = ""

            for linea in lineas:
                linea_upper = linea.upper()

                if "SECRETARIO GENERAL DEL SENADO" in linea_upper:
                    cargo_senado = "SECRETARIO GENERAL DEL SENADO"
                    nombre_senado_match = re.search(r"([A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑñ]+(?:\s+[A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑñ]+)+)", linea)
                    if nombre_senado_match:
                        nombre_senado = nombre_senado_match.group(1).strip()
                       

                elif "SECRETARIO GENERAL DE LA CÁMARA" in linea_upper:
                    cargo_camara = "SECRETARIO GENERAL DE LA CÁMARA"
                    nombre_camara_match = re.search(r"([A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑñ]+(?:\s+[A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑñ]+)+)", linea)
                    if nombre_camara_match:
                        nombre_camara = nombre_camara_match.group(1).strip()
                      
            if nombre_senado and cargo_senado:
                metadatos["directores"]["senado"]["nombre"] = nombre_senado
                metadatos["directores"]["senado"]["cargo"] = cargo_senado
            if nombre_camara and cargo_camara:
                metadatos["directores"]["camara"]["nombre"] = nombre_camara
                metadatos["directores"]["camara"]["cargo"] = cargo_camara
        else:
            print("No se pudo encontrar el final de la sección de directores.")
    else:
        print("No se encontró la palabra 'DIRECTORES:'")


    # --- Tipo de Documento / Descripción (Regex + spaCy) ---
    patron_tipo_documento = r"(?i)(COMENTARIOS)(.*?)(AL PROYECTO DE LEY NÚMERO \d+ DE \d+ SENADO)?(\".*?\")"
    doc_spacy = nlp(texto_completo)
    for oracion in doc_spacy.sents: #itera sobre las oraciones
        match = re.search(patron_tipo_documento, oracion.text, re.IGNORECASE | re.DOTALL)
        if match:
            metadatos["tipo_documento"] = match.group(1).strip() # primer grupo
            metadatos["subtitulo"] = match.group(2).strip()
            metadatos["subtitulo_1"] = match.group(4).strip() if match.group(4) else match.group(2).strip() #cuarto grupo
            break

    return metadatos

In [32]:
def extraer_con_regex(patron, texto):
    coincidencia = re.search(patron, texto, re.IGNORECASE)
    if coincidencia:
        return coincidencia.group(1).replace(" ", "") if len(coincidencia.groups()) == 1 else tuple(grupo.replace(" ", "") for grupo in coincidencia.groups())
    return None

def preprocesar_texto(texto):

    texto = texto.replace("-\n", "")
    texto = texto.replace("\n", " ")
    texto = re.sub(r"\s+", " ", texto)
    texto = texto.strip()
    return texto


def romano_a_entero(romano):
    valores = {'M': 1000, 'CM': 900, 'D': 500, 'CD': 400, 'C': 100, 'XC': 90,
               'L': 50, 'XL': 40, 'X': 10, 'IX': 9, 'V': 5, 'IV': 4, 'I': 1}
    entero = 0
    i = 0
    while i < len(romano):
        if i + 1 < len(romano) and romano[i:i+2] in valores:
            entero += valores[romano[i:i+2]]
            i += 2
        else:
            entero += valores[romano[i]]
            i += 1
    return entero

In [33]:
# --- Ejemplo de Uso ---
nlp = spacy.load("es_core_news_lg") 
matcher = Matcher(nlp.vocab)


texto_prueba = """
REPUBLICA
DE
COLOMBIA
CONGRESO DELAREPULICA
COL
GACETA
CONGRESO
SENADO Y CAMARA
Articulo3 6,L e y 5ade1 992)
IMPRENTA NACIONAL DE COLOMBIA
www.imprenta. gov.co
I S S N  0 1 2 3  -  9 0 6 6
AÑO XXV - N° 110
Bogotá, D. C., lunes, 28 de marzo de 2016
EDICION DE 64 PAGINAS
DIRECTORES:
GREGORIO ELJACH PACHECO
JORGE HUMBERTO MANTILLA SERRANO
SECRETARIO GENERAL DEL SENADO
SECRETARIO GENERAL DE LA CAMARA
www.secretariasenado.gov.co
www.camara.gov.co
RAMA LEGISLATIVA DEL PODER PUBLICO
SENADO DE LA REPUBLICA
COMENTARIOS
COMENTARIOS AL PROYECTO DE LEY NÚMERO 97 DE 2015 SENADO
"por la cual se prohibe la producción, comercialización, exportación, importación
y distribución de cualquier variedad de asbesto en Colombia"
COMENTARIOS DE LA ASOCIACION COLOMBIANA DE FIBRAS - ASCOLFIBRAS -.
RESEÑA respecto a la diferencia entre el crisotilo  
y los asbestos anfiboles
"El asbesto' no es en si un mineral. Es un término colectivo que: se da a an grupo. de minerales cuyos cristales ocurren en forma fibrosa. El término asbesto" fue adoptado como una identificacion comercial. 
Los seis minerales comunmente referidos como asbesto vienen de dos grupos distintivos de minerales. Un grupo es conocido como serpentina (crisotilo O asbesto blanco); mientras que el otro grupo es el del los anfiboles (amosita asbesto café; crocidolita asbesto azul; antofilita; tremolita; actinolita). Mientr as que mb DO son ales  atos, los dos grupos son quimica mineralogicamente diferen
"""

# Prueba con el texto de prueba
metadatos_prueba = extraer_metadatos_gaceta(texto_prueba)
if metadatos_prueba:
    print(metadatos_prueba)
else:
    print("No se pudieron extraer los metadatos del texto de prueba.")

{'issn': '0123-9066', 'fecha': '28/03/2016', 'entidades': ['SENADO'], 'anio_romano': 'XXV', 'anio': 25, 'numero_publicacion': 110, 'directores': {'senado': {}, 'camara': {}}, 'tipo_documento': 'COMENTARIOS', 'subtitulo': 'COMENTARIOS AL PROYECTO DE LEY NÚMERO 97 DE 2015 SENADO', 'subtitulo_1': '"por la cual se prohibe la producción, comercialización, exportación, importación y distribución de cualquier variedad de asbesto en Colombia"'}
